# This Notebook is for reviewing Cleaned EEG files and making edits to them

## Imports and Config settings

In [ ]:
from pathlib import Path

import pandas as pd
import mne

In [ ]:
mne.viz.set_browser_backend("qt")
mne.set_log_level("WARNING")

## Load (Or Create) a Tracking Sheet

In [ ]:
QCR_FNAME = Path.cwd() / "LISTEN-PHONEMES-EEG-QCR-Log.csv"
RECOMPUTE = False

deriv_dir = Path("/Volumes/UBUNTU18/USC/listen/derivatives/pylossless/run-01")
assert deriv_dir.exists()

QCR_DIR = Path("/Volumes/UBUNTU18/USC/listen/derivatives/QCR/run-01")
QCR_DIR.mkdir(exist_ok=True, parents=True)

sub_dirs = [sub_dir for sub_dir in deriv_dir.iterdir() if sub_dir.is_dir()]

if not QCR_FNAME.exists() or RECOMPUTE:
    filepaths = sub_dirs
    filenames = [fpath.name for fpath in filepaths]
    df = pd.DataFrame({"file" : filenames, "filepath": filepaths})
    df["status"] = pd.NA
    df["reviewer"] = pd.NA
    df["exclude"] = pd.NA
    df["notes"] = pd.NA
    df.to_csv(QCR_FNAME, index=False)
# Load the sub_dirs from the CSV file
qcr_df = pd.read_csv(QCR_FNAME)
qcr_df.head(10)

## Load the EEG Files

In [ ]:
fpath = "path/to/your/file.fif"
raw = mne.io.read_raw_fif("path/to/your/file.fif")
raw

## Review the EEG data

In [ ]:
fig = raw.plot_sensors(show_names=True, show=False)

In [ ]:
raw.plot(theme="light")

In [ ]:
epochs = mne.make_fixed_length_epochs(raw, duration=1, preload=True)
psd = epochs.drop_channels(epochs.info["bads"]).set_eeg_reference("average").compute_psd(fmin=2, fmax=50, method="welch")
del epochs
fig = psd.plot(show=False)

## Update the tracking sheet with your notes about the Data

In [ ]:
def update_qcr(
    qcr_df,
    fpath,
    *,
    status="Done",
    notes="Notes on the file",
    exclude=False,
    reviewer="SH",    
):
    filepath = Path(fpath).parent
    row_idx = qcr_df[qcr_df["filepath"] == filepath].index[0]
    if row_idx < 0:
        raise ValueError(f"Filepath {filepath} not found in QCR DataFrame.")
    qcr_df.astype({"status": "object", "notes": "object"})
    qcr_df.at[row_idx, "status"] = status
    qcr_df.at[row_idx, "notes"] = notes
    qcr_df.at[row_idx, "exclude"] = exclude
    qcr_df.at[row_idx, "reviewer"] = reviewer
    qcr_df.to_csv(QCR_FNAME, index=False)
    return pd.read_csv(QCR_FNAME)

In [ ]:
qcr_df = update_qcr(
    qcr_df,
    fpath,
    status="Done.",
    notes="Good.",
    exclude=False,
    reviewer="BK",
)
qcr_df.tail(10)

## Save the edited EEG file

In [ ]:
def save_qcr_raw(raw):
    fpath = raw.filenames[0]
    sub_dir = fpath.parent.name
    fname = fpath.name
    out_dir = QCR_DIR / sub_dir
    assert QCR_DIR.exists()
    out_dir.mkdir(exist_ok=True)
    out_fpath = out_dir / fname
    raw.save(out_fpath)
    return out_fpath

In [ ]:
save_qcr_raw(raw)